In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
df_agent = pd.read_csv('assessment_data/AGENT.csv')
df_main = pd.read_csv('assessment_data/MAIN.csv')
df_b = pd.read_csv('assessment_data/B.csv')
df_c = pd.read_csv('assessment_data/C.csv')  
df_d = pd.read_csv('assessment_data/D.csv')
df_e = pd.read_csv('assessment_data/E.csv')

Since we are only interested in  the Inforceagents under the agency distribution channel. We will filter the agent data first

In [4]:
print(len(df_agent))
df_agent = df_agent[(df_agent['AGT_CHANNEL'] == 'AGENCY') & (df_agent['AGT_STATUS'] == 'INFORCE')]
print(len(df_agent))
agent_code_list = df_agent['AGT_CD'].unique().tolist()

4812
3231


In [5]:
#filter out the agents that are inforce and have channel as AGENCY
print(len(df_main))
df_main = df_main[df_main['AGT_CD'].isin(agent_code_list)]
print(len(df_main))
df_main

598269
495854


,POLICY_NO,AGT_CD,S_BASE,RIDER_CD,COVERAGE_TYPE_IND,INS_UNIQUE_ID,PURCHASE_DT,PROD_CAT,PRODUCT_CD,ANP_RIDER,VIT_MBR_IND,PROD_SUBCAT_IND_1,PROD_SUBCAT_IND_2
0,1,3640,LAT,10102.0,R,P001664420,2020-03-26,NaN,ACC0,0.00,0,1,0
1,1,3640,LAT,10103.0,R,P001664420,2020-03-26,NaN,AHI0,0.00,0,1,1
2,1,3640,LAT,10108.0,R,P001664420,2020-03-26,NaN,VIT1,0.00,0,1,0
3,1,3640,LAT,10200.0,R,P001664420,2020-03-26,NaN,ART0,120.00,0,1,0
4,1,3640,LAT,10107.0,R,P001664420,2020-03-26,NaN,ART2,0.00,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
598255,551619,4212,LA,10100.0,B,P010373499,2019-09-13,NaN,MER1,88.30,0,1,0
598256,583668,4212,LA,10100.0,B,P002828561,2020-08-05,NaN,MER2,117.80,0,1,0
598257,583668,4212,LA,10101.0,R,P002828561,2020-08-05,NaN,APH1,29.60,0,1,0
598267,589062,3073,LA,10100.0,B,P010685096,2020-09-22,NaN,MER2,103.46,0,1,0


In [6]:
df_main['S_BASE'].value_counts()

S_BASE
LA      349418
LAT     130115
POLA     16321
Name: count, dtype: int64

Here I Assume LAT == LA Takaful , then we replace the value for joining the table to main

In [7]:
df_b['CHAR_VALUE2'] = np.where(df_b['CHAR_VALUE2'] == 'LA TAKAFUL','LAT', df_b['CHAR_VALUE2'])
df_d['CHAR_VALUE2'] = np.where(df_d['CHAR_VALUE2'] == 'LA TAKAFUL','LAT', df_d['CHAR_VALUE2'])

In [8]:
#rename the column to left join
df_main = df_main.rename(columns = {'PRODUCT_CD':'PURPOSE_CODE'})
df_main_b = pd.merge(df_main, df_b[['CHAR_VALUE','CHAR_VALUE2','PURPOSE_CODE']], how='left', left_on=['PURPOSE_CODE','S_BASE'], right_on=['PURPOSE_CODE','CHAR_VALUE2'])
df_main_b = df_main_b.rename(columns = {'CHAR_VALUE':'CHAR_VALUE_B',
                                        'CHAR_VALUE2':'CHAR_VALUE2_B'})
#take the one is null
df_main_c = pd.merge(df_main_b, df_c[['CHAR_VALUE','PURPOSE_CODE']], how='left', left_on=['PURPOSE_CODE'], right_on=['PURPOSE_CODE'])
df_main_c = df_main_c.rename(columns = {'CHAR_VALUE':'CHAR_VALUE_C'})
df_main_d = pd.merge(df_main_c, df_d[['CHAR_VALUE','CHAR_VALUE2','PURPOSE_CODE']], how='left', left_on=['PURPOSE_CODE','S_BASE'], right_on=['PURPOSE_CODE','CHAR_VALUE2'])
df_main_d = df_main_d.rename(columns = {'CHAR_VALUE':'CHAR_VALUE_D',
                                        'CHAR_VALUE2':'CHAR_VALUE2_D'})
df_main_e = pd.merge(df_main_d, df_e[['CHAR_VALUE','PURPOSE_CODE']], how='left', left_on=['PURPOSE_CODE'], right_on=['PURPOSE_CODE'])
df_main_e = df_main_e.rename(columns = {'CHAR_VALUE':'CHAR_VALUE_E'})

In [11]:
df_main = df_main_e

In [ ]:
#Get the main need from the main_need sql logic
df_main['MAIN_NEED'] = np.where(df_main['S_BASE'] == 'POLA', 'Accident',
                                np.where(df_main['CHAR_VALUE_D'].notna(), df_main['CHAR_VALUE_D'].str.strip(), #D
                                         np.where(df_main['CHAR_VALUE_E'].notna(), df_main['CHAR_VALUE_E'].str.strip(), #E
                                                  np.where((df_main['CHAR_VALUE_B'].notna()) & (df_main['CHAR_VALUE_B']!='Medical'), df_main['CHAR_VALUE_B'].str.strip(), #B
                                                           np.where((df_main['CHAR_VALUE_B'].notna()) & (df_main['CHAR_VALUE_B']=='Medical'), 'Other Medical', #B
                                                                    np.where((df_main['CHAR_VALUE_C'].notna()) & (df_main['CHAR_VALUE_C']!='Medical'), df_main['CHAR_VALUE_C'].str.strip(), #C
                                                                             np.where((df_main['CHAR_VALUE_C'].notna()) & (df_main['CHAR_VALUE_C']=='Medical'), 'Other Medical', None))))))) #C

In [21]:
df_main['MAIN_NEED'].value_counts()

MAIN_NEED
Medical             90069
Life                87713
Disability          78354
Savers              77400
Payor/Waiver        53946
Critical Illness    44039
Accident            35009
Income              18958
Other Medical        7195
Endowment            3171
Name: count, dtype: int64

In [ ]:
#Fill PROD_CAT based on the product_category sql logic
df_main['PROD_CAT'] = np.where(df_main['PROD_CAT'].notna(),
                               df_main['PROD_CAT'], 
                               np.where((df_main['PROD_SUBCAT_IND_1'] == 1) & ((df_main['PROD_SUBCAT_IND_2'] == 1) | df_main['MAIN_NEED'].isin(['Medical','Other Medical','Critical Illness'])),
                                        'Cat 2',
                                        'Cat 3'))
df_main['PROD_CAT'].value_counts()

PROD_CAT
Cat 3    360331
Cat 2    123756
Cat 1     11767
Name: count, dtype: int64

In [26]:
df_main = df_main.drop(columns = ['CHAR_VALUE_B',
                                  'CHAR_VALUE_C',
                                  'CHAR_VALUE_D',
                                  'CHAR_VALUE_E',
                                  'CHAR_VALUE2_B',
                                  'CHAR_VALUE2_D'])
df_main

,POLICY_NO,AGT_CD,S_BASE,RIDER_CD,COVERAGE_TYPE_IND,INS_UNIQUE_ID,PURCHASE_DT,PROD_CAT,PURPOSE_CODE,ANP_RIDER,VIT_MBR_IND,PROD_SUBCAT_IND_1,PROD_SUBCAT_IND_2,MAIN_NEED
0,1,3640,LAT,10102.0,R,P001664420,2020-03-26,Cat 2,ACC0,0.00,0,1,0,Critical Illness
1,1,3640,LAT,10103.0,R,P001664420,2020-03-26,Cat 2,AHI0,0.00,0,1,1,Income
2,1,3640,LAT,10108.0,R,P001664420,2020-03-26,Cat 3,VIT1,0.00,0,1,0,Life
3,1,3640,LAT,10200.0,R,P001664420,2020-03-26,Cat 3,ART0,120.00,0,1,0,Savers
4,1,3640,LAT,10107.0,R,P001664420,2020-03-26,Cat 3,ART2,0.00,0,1,0,Accident
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495849,551619,4212,LA,10100.0,B,P010373499,2019-09-13,Cat 2,MER1,88.30,0,1,0,Medical
495850,583668,4212,LA,10100.0,B,P002828561,2020-08-05,Cat 2,MER2,117.80,0,1,0,Medical
495851,583668,4212,LA,10101.0,R,P002828561,2020-08-05,Cat 2,APH1,29.60,0,1,0,Medical
495852,589062,3073,LA,10100.0,B,P010685096,2020-09-22,Cat 2,MER2,103.46,0,1,0,Medical
